# Семинар 11. Автоэнкодеры на PyTorch. Препарирование scVI

## Домашнее задание

Изучите документацию scVI: https://docs.scvi-tools.org/en/stable/api/reference/scvi.model.SCVI.html.

Какие параметры модели можно регулировать? За что они отвечают? Попробуйте поменять их значения и исследуйте, как они влияют на качество коррекции батч-эффекта на использованном нами датасете.

Какие параметры сильно влияют на обучение? Какие дают наилучший результат?

In [ ]:
!pip install torch torchvision
!pip install scanpy

!pip install --quiet scvi-colab
from scvi_colab import install
install()

import os
os.kill(os.getpid(), 9)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 34.4 MB/s 
     |████████████████████████████████| 88 kB 8.8 MB/s 
     |████████████████████████████████| 9.4 MB 50.9 MB/s 
     |████████████████████████████████| 96 kB 4.9 MB/s 
     |████████████████████████████████| 295 kB 57.7 MB/s 
     |████████████████████████████████| 965 kB 49.2 MB/s 
     |████████████████████████████████| 1.1 MB 54.0 MB/s 
     |████████████████████████████████| 63 kB 2.3 MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82829 sha256=0dd593c9abba8d03940ca85cb9af4eb3ecd71899664d9dc4890c5d32b20a35d3
  Stored in directory: /root/.cache/pip/wheels/a9/3a/67/06a8950e053725912e6a8c42c4a3a241410f6487b8402542ea
  Created wheel for pynndescent: filename=pynndescent-0.5.8-py3-none-any.whl si

     |████████████████████████████████| 237 kB 37.8 MB/s 
     |████████████████████████████████| 51 kB 867 kB/s 
INFO     scvi-colab: Installing scvi-tools.                                                                        
INFO     scvi-colab: Install successful. Testing import.                                                           


INFO:pytorch_lightning.utilities.seed:Global seed set to 0


In [1]:
import scvi
import scanpy as sc
import numpy as np

sc.set_figure_params(figsize=(5, 5))

INFO:pytorch_lightning.utilities.seed:Global seed set to 0
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/utilities/warnings.py:53: LightningDeprecationWarning: pytorch_lightning.utilities.warnings.rank_zero_deprecation has been deprecated in v1.6 and will be removed in v1.8. Use the equivalent function from the pytorch_lightning.utilities.rank_zero module instead.
  new_rank_zero_deprecation(
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/utilities/warnings.py:58: LightningDeprecationWarning: The `pytorch_lightning.loggers.base.rank_zero_experiment` is deprecated in v1.7 and will be removed in v1.9. Please use `pytorch_lightning.loggers.logger.rank_zero_experiment` instead.
  return new_rank_zero_deprecation(*args, **kwargs)


In [2]:
#загружаем датасет
!wget -O PBMC_Satija.h5ad --no-check-certificate "https://docs.google.com/uc?export=download&id=1jW548g6ERFS0t7NywgyjRs6VaE5QwXbg&confirm=t"

--2022-12-10 16:10:52--  https://docs.google.com/uc?export=download&id=1jW548g6ERFS0t7NywgyjRs6VaE5QwXbg&confirm=t
Resolving docs.google.com (docs.google.com)... 142.250.4.102, 142.250.4.138, 142.250.4.139, ...
Connecting to docs.google.com (docs.google.com)|142.250.4.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-04-6s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/bt7hpvij8flve5onnluthh61na3vt3l3/1670688600000/08626740090461398144/*/1jW548g6ERFS0t7NywgyjRs6VaE5QwXbg?e=download&uuid=659dd1fb-2d39-4e41-a325-f388cff0a179 [following]
--2022-12-10 16:10:54--  https://doc-04-6s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/bt7hpvij8flve5onnluthh61na3vt3l3/1670688600000/08626740090461398144/*/1jW548g6ERFS0t7NywgyjRs6VaE5QwXbg?e=download&uuid=659dd1fb-2d39-4e41-a325-f388cff0a179
Resolving doc-04-6s-docs.googleusercontent.com (doc-04-6s-docs.googleusercontent.com)... 172.253.118.132, 240

Читаем датасет

In [3]:
adata = sc.read_h5ad("PBMC_Satija.h5ad")
adata.layers["counts"] = adata.X.copy()

Контроль дисперсии + фильтрация наиболее вариабельных генов

In [4]:
sc.pp.filter_genes(adata, min_counts=3)
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
adata.raw = adata
sc.pp.highly_variable_genes(
    adata,
    n_top_genes=1200,
    subset=True,
    layer="counts",
    flavor="seurat_v3",
    batch_key="donor"
)

Создаю метрику для оценки батч-корреляции


In [5]:
def loss_metric(adata, cluster_key, batch_key):
    
    sizes = adata.obs.groupby([batch_key, cluster_key]).size()
    props = sizes.groupby(level=1).apply(lambda x: 100 * x / x.sum()).reset_index()
    props = props.pivot(columns=cluster_key, index=batch_key).T
    
    # метрика
    metrics = props.assign(metric=lambda x: np.abs(x['P1'] - 33.3) + np.abs(x['P2'] - 33.3) + np.abs(x['P3']) - 33.3)
    print("\nBatch correlation loss metric:", metrics["metric"].mean())
    return metrics["metric"].mean()

In [7]:
import itertools

scvi.model.SCVI.setup_anndata(
    adata,
    layer="counts",
    batch_key="donor"
)

# параметры модели
param_dict = {"n_hidden": [128, 256, 512],
              "n_latent":[10,15, 20, 30],
               "dispersion":["gene", "gene-batch"],
              "n_layers": [1, 3, 5]
              }

params_set = itertools.product(*param_dict.values())
loss_metrics = []

for param_set in params_set:

  # инициализируем модель с указанными параметрами
  model = scvi.model.SCVI(adata, n_hidden = param_set[0], 
                          n_latent = param_set[1], 
                          dispersion = param_set[2],
                          n_layers = param_set[3])
  
  model.train(max_epochs=100)

  latent = model.get_latent_representation()
  adata.obsm["X_scVI"] = latent
  sc.pp.neighbors(adata, use_rep = "X_scVI")
  sc.tl.leiden(adata, key_added="leiden_scVI", resolution=0.5)
  sc.tl.umap(adata)

  print("\n\nModel params:", list(zip(param_dict.keys(), param_set)))
  sc.pl.umap(adata, color=["celltype.l1", "leiden_scVI", "donor"],
           title=["Cell type", "Clusters", "Batch"], wspace=0.4, frameon=False);
  loss_metrics.append(loss_metric(adata=adata, cluster_key="leiden_scVI", batch_key="donor"))

  print("\n")

Output hidden; open in https://colab.research.google.com to view.

In [41]:
results_dict = {k:v for k,v in list(zip(loss_metrics, list(params_set)))}

In [40]:
params_set = itertools.product(*param_dict.values())

In [43]:
dict(sorted(results_dict.items()))

{17.123593458201768: (256, 20, 'gene', 3),
 17.12917282577312: (512, 20, 'gene-batch', 5),
 17.424731412144098: (256, 10, 'gene-batch', 1),
 17.579494448124326: (512, 20, 'gene', 5),
 17.898208034064705: (256, 15, 'gene', 5),
 18.255344455764327: (128, 20, 'gene-batch', 3),
 18.346185665726235: (512, 10, 'gene-batch', 5),
 18.460420322815136: (256, 30, 'gene', 3),
 18.49231824426264: (512, 15, 'gene', 5),
 18.505259842107115: (512, 30, 'gene-batch', 5),
 18.615755842055425: (512, 10, 'gene', 5),
 18.664956440291476: (128, 30, 'gene-batch', 3),
 18.681633844786255: (128, 15, 'gene-batch', 5),
 18.794710373077564: (256, 10, 'gene-batch', 3),
 18.902818409707482: (256, 30, 'gene-batch', 3),
 18.95918623745648: (512, 10, 'gene', 3),
 19.034456598617485: (256, 20, 'gene-batch', 3),
 19.079869899454042: (128, 10, 'gene-batch', 5),
 19.70102420809968: (256, 20, 'gene', 5),
 19.82295299344182: (256, 20, 'gene-batch', 1),
 19.85784768333275: (128, 15, 'gene', 5),
 19.869201360165473: (256, 15, 

Так, мы видим, что самый низкий коэффициент ошибки у автоэнкодера с параметрами 256 нейронов в скрытом слое, с латентным пространством = 20, параметром дисперсии в гене (а не в гене по батчу), и 3 скрытыми слоями. Немного контринтуитивно, что принцип "чем больше слоёв/нейронов, тем лучше" не сработал, но результаты есть результаты :)